# December 17, 2022
https://adventofcode.com/2022/day/17

In [1]:
# Too low: 3132, 3133
# Too high: 5256

In [2]:
test = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"

In [3]:
fn = "data/17.txt"
with open(fn, "r") as file:
    puz = file.readline()

In [4]:
# Tower is oriented [x,y] with [0,0] being left/bottom
class Tower:
    def __init__(self, jets, width=7, retain_rows = 0):
        # Note: bank() method only works for width 4-7!
        self.width = width

        # track which gust comes next
        self.jets = jets
        self.jet_pos = 0

        # instantiate map with floor
        # self.map is recorded from bottom up
        self.map = [ ['-']*self.width ]
        self.height = 0
        self.banked_height = 0
        self.retain_rows = retain_rows
        if retain_rows == 0:
            self.banking = False
        else:
            self.banking = True
            
        
        # track current active block
        # 0Flat, Cross, 2Ell, 3Pipe, 4Tuba
        self.next = 0
        self.block_count = 0
        self.spawn_x = 2
        self.spawn_dy = 4
        self.new_block()



    def check(self, x, y):
       # print("Checking",x,y)
        # walls are closed
        if x < 0 or x >= self.width:
            return False

        # open above
        if y > self.height:
            return True

        # otherwise check map
       # print("Looking at map")
       # print(self.map[y][x])
        return  (self.map[y][x]=='.') # using this for blank

    def bank(self):

        if self.height > self.retain_rows:
            to_bank = self.height - self.retain_rows
            self.map = self.map[-(1+self.retain_rows):]
            self.height = self.retain_rows
            self.banked_height += to_bank

            if self.block is not None:
                #print("Block at", self.block.x, self.block.y, "Banking", to_bank)
                self.block.y -= to_bank
                #print("Mapped to", self.block.x, self.block.y)
                # if block is below the retain line, we're assuming it doesn't matter
                if self.block.y <= 0:
                  #  pixels = self.block.pixels()
                  #  for xy in pixels:
                  #      if xy[1] > 0:
                  #          print(xy)
                  #          self.solidify(xy[0], xy[1], self.block.char)
                    self.new_block()
        else:
            pass
        return
    

    def new_block(self):
        y = self.height + self.spawn_dy
        if self.next == 0:
            self.block = Flat(self.spawn_x, y, self)
        elif self.next == 1:
            self.block = Cross(self.spawn_x,y, self)
        elif self.next == 2:
            self.block = Ell(self.spawn_x,y, self)
        elif self.next == 3:
            self.block = Pipe(self.spawn_x,y, self)
        elif self.next == 4:
            self.block = Tuba(self.spawn_x,y, self)
        self.next = (self.next + 1) % 5
        self.block_count += 1

    def gust(self):
        jet = self.jets[self.jet_pos]
        if jet == "<":
            if self.block.check_left():
                self.block.x -= 1
        else: # jet == ">":
            if self.block.check_right():
                self.block.x += 1
        self.jet_pos = (self.jet_pos + 1) % len(self.jets)
        if self.jet_pos == 0 and self.banking:
            self.bank()


    def drop(self):
        # return True if block dropped, False if it's stuck
        if self.block.check_down():
            self.block.y -= 1
            if self.block.y == 0: # assume the block falls into oblivion and doesn't affect tower height
                self.new_block()
                return False
            return True
        else:
            pixels = self.block.pixels()
            for xy in pixels:
                self.solidify(xy[0], xy[1], self.block.char)
            if not any( [x == '.' for x in self.map[-1]] ):
                print("Flat line:", self.block_count)
            self.new_block()
            return False

    def solidify(self, x, y, char):
        # add to tower height as needed
        dy = y - self.height
        for i in range(dy):
            self.map += [ ['.']*self.width ]
            self.height += 1

        self.map[y][x] = char

    def __str__(self):
        addendum = []
        out = []

        if self.block is not None:
            dy = self.block.y + (self.block.height-1) - self.height
            pixels = self.block.pixels()
            for y in range(dy):
                addendum.append( ['.']*self.width )

            for xy in pixels:
                if xy[1] > self.height:
                    # add to addendum
                    y2 = xy[1]-1 - self.height
                    addendum[y2][xy[0]] = '@'
                else:
                    self.map[xy[1]][xy[0]] = '@'

            for line in addendum[::-1]:
                out.append( "".join(line) )
        
        for line in self.map[::-1]:
            out.append( "".join(line) )

        if self.block is not None:
            # reset the map's block spaces
            for xy in pixels:
                if xy[1] <= self.height:
                    self.map[xy[1]][xy[0]] = '.'
                
        return "\n".join(out)
    
    def __repr__(self):
        return self.__str__(self)

class Block:
    pass

class Flat(Block):
    # refpoint is left-most
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Flat"
        self.height = 1
        self.char = "A"

    def check_right(self):
        return self.tower.check(self.x+4, self.y)
    def check_left(self):
        return self.tower.check(self.x-1, self.y)
    def check_down(self):
        for x in range(self.x, self.x+4):
            if not self.tower.check(x, self.y-1):
                return False
        return True
    def pixels(self):
        return [ [self.x, self.y], [self.x+1, self.y], [self.x+2, self.y], [self.x+3, self.y] ]

class Cross(Block):
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Cross"
        self.height = 3
        self.char = "B"


    def check_right(self):
        return (self.tower.check(self.x+2, self.y)
                and self.tower.check(self.x+3, self.y+1)
                and self.tower.check(self.x+2, self.y+2))
    def check_left(self):
        return (self.tower.check(self.x, self.y)
                and self.tower.check(self.x-1, self.y+1)
                and self.tower.check(self.x, self.y+2))
    def check_down(self):
        return (self.tower.check(self.x, self.y)
                and self.tower.check(self.x+1, self.y-1)
                and self.tower.check(self.x+2, self.y))
    def pixels(self):
        return [ [self.x+1, self.y], [self.x, self.y+1], [self.x+1, self.y+1], [self.x+2, self.y+1], [self.x+1, self.y+2] ]

class Ell(Block):
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Ell"
        self.height = 3
        self.char = "C"

    def check_right(self):
        return (self.tower.check(self.x+3, self.y)
                and self.tower.check(self.x+3, self.y+1)
                and self.tower.check(self.x+3, self.y+2))
    def check_left(self):
        return (self.tower.check(self.x-1, self.y)
                and self.tower.check(self.x+1, self.y+1)
                and self.tower.check(self.x+1, self.y+2))
    def check_down(self):
        return (self.tower.check(self.x, self.y-1)
                and self.tower.check(self.x+1, self.y-1)
                and self.tower.check(self.x+2, self.y-1))
    def pixels(self):
        return [ [self.x, self.y], [self.x+1, self.y], [self.x+2, self.y], [self.x+2, self.y+1], [self.x+2, self.y+2] ]

class Pipe(Block):
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Pipe"
        self.height = 4
        self.char = "D"


    def check_right(self):
        return (self.tower.check(self.x+1, self.y)
                and self.tower.check(self.x+1, self.y+1)
                and self.tower.check(self.x+1, self.y+2)
                and self.tower.check(self.x+1, self.y+3))
    def check_left(self):
        return (self.tower.check(self.x-1, self.y)
                and self.tower.check(self.x-1, self.y+1)
                and self.tower.check(self.x-1, self.y+2)
                and self.tower.check(self.x-1, self.y+3))
    def check_down(self):
        return self.tower.check(self.x, self.y-1)
    def pixels(self):
        return [ [self.x, self.y], [self.x, self.y+1], [self.x, self.y+2], [self.x, self.y+3] ]

class Tuba(Block): # Two-buh-two
    # refpoint is bottom-left (empty space)
    def __init__(self, x, y, tower):
        self.x = x
        self.y = y
        self.tower = tower
        self.type = "Tuba"
        self.height = 2
        self.char = "E"


    def check_right(self):
        return (self.tower.check(self.x+2, self.y)
                and self.tower.check(self.x+2, self.y+1))
    def check_left(self):
        return (self.tower.check(self.x-1, self.y)
                and self.tower.check(self.x-1, self.y+1))
    def check_down(self):
        return (self.tower.check(self.x, self.y-1)
                and self.tower.check(self.x+1, self.y-1))
    def pixels(self):
        return [ [self.x, self.y], [self.x+1, self.y], [self.x, self.y+1], [self.x+1, self.y+1] ]


In [5]:
def play_game( tower, until=2023, verbose=0 ):
    while tower.block_count < until:
        #print(tower.jets[tower.jet_pos], tower.block.type)
        if verbose >= 2:
            print("\n", tower.jets[tower.jet_pos], tower.block.type)
        tower.gust()
        if verbose >= 4:
            print(tower)
        success = tower.drop()
        if (not success and verbose >=1) or verbose >= 3:
            print("\n", tower, "\n", sep="")

### Part 1

In [6]:
test_tower = Tower(test)
play_game(test_tower, 2023)
true_ans = test_tower.height + test_tower.banked_height
print(true_ans)

3068


In [7]:
puz_tower = Tower(puz, )
play_game(puz_tower, 2023)
part1 = puz_tower.height + puz_tower.banked_height
print(part1)

Flat line: 136
3181


In [124]:
# 50 rows appears to be enough memory ---
# but then we have C*2^50 maps to memoize :(
for i in [50,100,200,500,1000]:
    puz_tower = Tower(puz, retain_rows=i)
    play_game(puz_tower, 500000)
    ans = puz_tower.height + puz_tower.banked_height
    print(i, ans)

50 785238
100 785238
200 785238
500 785238
1000 785238


In [127]:
# Flat lines are too rare to memoize based on that...
puz_tower = Tower(puz, retain_rows=50)
play_game(puz_tower, 1000000)
ans = puz_tower.height + puz_tower.banked_height
print(ans)

Flat line: 136
1570454


In [8]:
1570454*1000000, 1514285714288

(1570454000000, 1514285714288)

In [37]:
test_tower = Tower(test)
play_game(test_tower, 1000000000000)
print(test_tower.height + test_tower.banked_height)

KeyboardInterrupt: 

In [38]:
test_tower.block_count, 1000000000000/test_tower.block_count

(1736524, 575863.0459469607)

In [39]:
575863/60

9597.716666666667